In [1]:
#dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

#import API key
from config import g_key

#access maps with unique API key
gmaps.configure(api_key=g_key)

In [2]:
#import cities.csv file into a dataframe
cities_df = pd.read_csv("cities.csv")

#get rid of 'City_ID' column
cities_df = cities_df[["City", "Cloudiness", "Country", 
                       "Date", "Humidity", "Lat", "Lng", "Max Temp", "Wind Speed"]]
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Omboue,94,GA,1579799435,91,-1.57,9.26,78.89,2.59
1,Port-Gentil,40,GA,1579799195,70,-0.72,8.78,89.60,5.82
2,Jamestown,1,US,1579799437,61,42.10,-79.24,37.40,4.70
3,Mumford,70,GH,1579799438,65,5.26,-0.76,83.86,9.08
4,Gamba,98,GA,1579799301,82,-2.65,10.00,80.91,4.81
5,Anloga,10,GH,1579799440,66,5.79,0.90,87.80,6.93
6,Takoradze,86,GH,1579799285,70,4.88,-1.76,83.66,8.08
7,Winneba,67,GH,1579799443,70,5.35,-0.62,87.80,11.41
8,Teshie,44,GH,1579799235,70,5.58,-0.11,87.80,11.41


In [3]:
#try this on the larger dataframe
#cities_nan = cities_df[cities_df.isna().any(axis=1)]
#cities_nan
#if there are nan values, then save by coding cities_df = cities_df.dropna()

In [4]:
#create heatmap based on humidity

#store latitude and longitude into the variable 'coordinates'
coordinates = cities_df[["Lat", "Lng"]]

#convert humidity to float; all data read from a csv is read as a string
humidity = cities_df["Humidity"].astype(float)

In [7]:
#plot heatmap
fig = gmaps.figure()

#create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

#add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))